In [57]:
import pandas as pd
from pathlib import Path
import io, requests

All columns from the kaggle dataset:

`Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')`

In [59]:
# ---------------------------------------------------------------------
# 1.  Load Kaggle game metadata (date, season, id)
# ---------------------------------------------------------------------
CSV_DIR   = Path("data/csv")
DATA_DIR  = Path("data")

game_df = pd.read_csv(CSV_DIR / "game.csv",
                      usecols=["game_id", "game_date", "season_id", "wl_home", 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'plus_minus_home', 'team_id_away',
       'team_name_away', 'matchup_away', 
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'plus_minus_away'])
game_df["date_game"] = pd.to_datetime(game_df["game_date"]).dt.date

In [60]:
# ---------------------------------------------------------------------
# 2.  Load Kaggle line scores (already 1 row per TEAM per GAME)
# ---------------------------------------------------------------------
line_df = pd.read_csv(CSV_DIR / "line_score.csv",
                      usecols=["game_id", "team_abbreviation_home", "pts_home"])

# bring in the date + season
kaggle_long = line_df.merge(game_df,
                            on="game_id", how="inner")

In [61]:
# ---------------------------------------------------------------------
# 3.  Load & reshape FiveThirtyEight Elo → long format
# ---------------------------------------------------------------------
elo_538_raw = pd.read_csv(DATA_DIR / "nbaallelo.csv")
elo_538_raw["date_game"] = pd.to_datetime(elo_538_raw["date_game"]).dt.date

elo_team1 = (
    elo_538_raw[["date_game", "team_id", "elo_i", "elo_n"]]
      .rename(columns={"team_id": "TEAM_ABBREVIATION",
                       "elo_i": "elo_pre_538",
                       "elo_n": "elo_post_538"})
)
elo_team2 = (
    elo_538_raw[["date_game", "opp_id", "opp_elo_i", "opp_elo_n"]]
      .rename(columns={"opp_id": "TEAM_ABBREVIATION",
                       "opp_elo_i": "elo_pre_538",
                       "opp_elo_n": "elo_post_538"})
)

elo_538_long = pd.concat([elo_team1, elo_team2], ignore_index=True)

In [62]:
# ---------------------------------------------------------------------
# 4.  Reconcile historical team codes (minimal starter dict)
# ---------------------------------------------------------------------
alias = {
    "NJN": "BKN", "BRK": "BKN",   # Nets
    "NOH": "NOP", "NOK": "NOP",   # Pelicans
    "CHH": "CHA",                 # Old Hornets
    "SEA": "OKC",                 # Sonics → Thunder
}
kaggle_long["team_abbreviation_home"] = kaggle_long["team_abbreviation_home"].replace(alias)
kaggle_long.rename(columns={"team_abbreviation_home": "TEAM_ABBREVIATION"}, inplace=True)
elo_538_long["TEAM_ABBREVIATION"] = elo_538_long["TEAM_ABBREVIATION"].replace(alias)

In [63]:
# ---------------------------------------------------------------------
# 5.  Merge on date + franchise code
# ---------------------------------------------------------------------
merged = kaggle_long.merge(
    elo_538_long,
    on=["date_game", "TEAM_ABBREVIATION"],
    how="inner"
)

In [64]:
# ---------------------------------------------------------------------
# 6.  Save the merged DataFrame to a CSV file
# ---------------------------------------------------------------------
merged.to_csv("data/raw/combined_kaggle_538_elo.csv", index=False)
